In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "SNHU1234"

shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Justin Starr'))),
    html.Hr(),
    
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    html.Div(
        [
            dcc.RadioItems(
                id='filter',
                options=[
                    {'label': "Water Rescue", 'value': "Water"},
                    {'label': "Mountain or Wilderness Rescue", 'value': "Mountain"},
                    {'label': "Disaster Rescue or Individual Tracking", 'value': "Disaster"},
                    {'label': "Reset", 'value': "Reset"} 
                ],
                value="Reset" #starting value for the radio buttons
            )    
        ]
    ),   
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here  
        editable=False,
        filter_action="native",
        filter_options={"placeholder_text": "Filter column..."},
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    # Unique Identifiers
    html.Div(children=[ 
        html.H1(children='Hello Justin,'),
        
        html.Div(children='''
            Justin Starr SNHU CS-340 Client/Server Application.
        ''')
    ]),
                
    html.Br(),
    html.Hr(),
   
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
            children=[
                html.Div([
                    # Dropdown menu for the pie chart
                    dcc.Dropdown(
                        id='piechart_dropdown',
                        className='col s12 m6',
                        options=[
                            {'label': 'Breed', 'value': 'breed'},
                            {'label': 'Preferred Sex', 'value': 'sex_upon_outcome'},
                            {'label': 'Age of Animals in Weeks', 'value': 'age_upon_outcome_in_weeks'}
                        ],
                        value='breed', #starting value for the dropdown
                        multi=False,
                        clearable=False
                    )
                ],
                style={"width":"15%"}
                ),    
    
        # For the pie chart
        html.Div(        
            id='graph-id',
            className='col s12 m6',
        ),

        # For the geolocation map
        html.Div(
            id='map-id',
            className='col s12 m6',
        ),
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            height = 500, width = 500
        )    
    ])
])
#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id', 'data'),
               Output('datatable-id', 'columns')],
              [Input('filter', 'value')])

def update_dashoard(filter_type):    
    
    #Queries based on rescue type and preferred dog breeds
    
    #start case
    df = pd.DataFrame.from_records(shelter.read({}))
    
    # query for water rescue
    if (filter_type=="Water"):        
        df = pd.DataFrame(list(shelter.read({ '$and': [     
            {'$or': [{"breed": "Labrador Retriever Mix"}, 
                     {"breed": "Chesapeake Bay Retriever"}, 
                     {"breed": "Newfoundland"}]},
            {"sex_upon_outcome": "Intact Female"},
            {'$and': [{"age_upon_outcome_in_weeks": {'$gte': 26}}, 
                      {"age_upon_outcome_in_weeks": {"$lte": 156}}]}    
        ]})))
    
    # query for mountain or wilderness rescue
    elif (filter_type == "Mountain"):
        df = pd.DataFrame(list(shelter.read({ '$and': [     
            {'$or': [{"breed": "German Shepherd"}, 
                     {"breed": "Alaskan Malamute"}, 
                     {"breed": "Old English Sheepdog"},
                     {"breed": "Siberian Husky"},
                     {"breed": "Rottweiler"}]},
            {"sex_upon_outcome": "Intact Male"},
            {'$and': [{"age_upon_outcome_in_weeks": {'$gte': 26}}, 
                      {"age_upon_outcome_in_weeks": {"$lte": 156}}]}  
        ]})))    
    
    # query for disaster or individual tracking
    elif (filter_type == "Disaster"):
        df = pd.DataFrame(list(shelter.read({ '$and': [     
            {'$or': [{"breed": "Doberman Pinscher"}, 
                     {"breed": "German Shepherd"}, 
                     {"breed": "Golden Retriever"},
                     {"breed": "Bloodhound"},
                     {"breed": "Rottweiler"}]},
            {"sex_upon_outcome": "Intact Male"},
            {'$and': [{"age_upon_outcome_in_weeks": {'$gte': 20}}, 
                      {"age_upon_outcome_in_weeks": {"$lte": 300}}]} 
        ]})))    
    
    # query for resetting the data table by reading all documnets in the 
    # specified database and collection
    elif (filter_type == "Reset"):
        df = pd.DataFrame.from_records(shelter.read({}))   
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    df.drop(columns=['_id'],inplace=True)
    
    return (df.to_dict('records'), columns) 

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')],
)
def update_styles(selected_rows):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]

# Display the breeds of animal based on quantity represented in the data table
# Display the age in weeks of animals based on quantity represented in the data table
# Display the outcome type of animals based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('piechart_dropdown', 'value')])
def update_graphs(viewData, dropDownValue):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    df = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
            figure = px.pie(data_frame=df, names=dropDownValue, title="Preferred Animals")
        )    
    ]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    #FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can
    # be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]
    # Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
            center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])    
            ])    
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:13879/
